In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import html5lib

To start this whole thing out, I need to collect the urls from wikipedia that have contestant data. I do this for both the Bachelor and Bachelorette.

In [2]:
import numpy as np
bachelor_seasons = np.arange(9,21)

bachelor_urls = []
for season in bachelor_seasons:
    season_url = 'https://en.wikipedia.org/wiki/The_Bachelor_(season_{})'.format(season)
    bachelor_urls.append(season_url)

In [3]:
import numpy as np
bachelorette_seasons = np.arange(4,13)
bachelorette_seasons = np.insert(bachelorette_seasons,0,2)

bachelorette_urls = []
for season in bachelorette_seasons:
    season_url = 'https://en.wikipedia.org/wiki/The_Bachelorette_(season_{})'.format(season)
    bachelorette_urls.append(season_url)

Let's create a function that takes in a url and then creates a soup from the html. This will come in handy when we are collecting the data.

In [4]:
def makeSoup(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.content,"lxml")
    return soup

In [5]:
def dataCollector(soup):
    import re
    # make the soup
    this = makeSoup(soup)
    
    # find the right table
    tables = this.findChildren('table')
    table = tables[1]
    
    # turn the data into a workable format
    data   = [[td.text for td in row.select('td')]
             for row in table.findAll('tr')]
    
    # create the header row and the body
    header = ['name','age', 'hometown', 'occupation', 'elimination']
    body = data[1:]
    cols   =  zip(*body)
    
    # create a dict with the data
    tbl_d  = {name:col for name, col in zip(header,cols)}
    
    # extract the season number from the original soup
    number = re.findall(r'\d+', soup)
    
    # remove brackets
    num= ''.join(number)
    
    # create a new key of seasons
    tbl_d['season'] = [num] * len(tbl_d['age'])
    
    new_names = []
    for name in tbl_d['name']:
        cleaned_name = re.sub(r'\[\w+\]', ' ', name)
        new_names.append(cleaned_name)

    tbl_d['name'] = new_names
    
    # return dictionary
    return tbl_d

In [6]:
def frameMaker(urls):
    frames = []

    for url in urls:
        dictionary = dataCollector(url)
        frame = pd.DataFrame(dictionary)
        frames.append(frame)

    combined = pd.concat(frames)
    
    return combined.reset_index()

In [7]:
bachelorette_frame = frameMaker(bachelorette_urls)

In [42]:
bachelor_frame = frameMaker(bachelor_urls)

In [38]:
bachelorette_frame.tail(50)

,index,age,elimination,hometown,name,occupation,season
207,2,26,Eliminated in week 8,"Warsaw, Indiana","Benjamin ""Ben"" Higgins",Software Salesman,11
208,3,26,Eliminated in week 7,"Warwick, Rhode Island",Jared Haibon,Restaurant Manager,11
209,4,28,Eliminated in week 7,"Glasgow, Kentucky","Joseph ""Joe"" Bailey",Insurance Agent,11
210,5,28,Eliminated in week 7,"Granite Bay, California","Christopher ""Chris"" Strandburg",Dentist,11
211,6,26,Eliminated in week 6,"Falls Church, Virginia","Benjamin ""Ben"" Zorn",Fitness Coach,11
212,7,28,Eliminated in week 6,"Stilwell, Kansas",Tanner Tolbert,Auto Finance Manager,11
213,8,32,Eliminated in week 6,"Dacono, Colorado","John ""JJ"" Lane III",Former Investment Banker,11
214,9,31,Eliminated in week 5,"Ventura, California",Joshua Albers,Industrial Welder,11
215,10,28,Eliminated in week 5,"Elgin, Illinois",Justin Reich,Fitness Trainer,11
216,11,28,Quit in week 5,"Ramsey, New Jersey",Ian Thomson,Executive Recruiter,11


In [10]:
for name in bachelorette_frame['name']:
    first_name = name.split(" ")
    print(first_name[0])

Ian
Matthew
Chad
Lanny
Brad
Ryan
Sean
Rick
Ryan
Todd
Damon
Eliot
Harold
Marcus
Robert
Aaron
Andy
Anselm
Brian
Chris
Cory
Jeff
Justin
Keith
Trever
Jesse
Jason
Jeremy
Graham
Sean
Blaine
Brian
Frederick
Robert
Paul
Richard
Ronald
Christopher
Eric
Ryan
Brian
Chandler
Donato
Gregory
Jeffrey
Jonathan
Luke
Patrick
Patrick
Spero
Ed
Kiptyn
Reid
Wes
Jesse
Michael
Jake
Tanner
Robby
Mark
David
Juan
Mike
Brad
Tanner
Sasha
Brian
Julien
Mathue
Simon
Adam
Bryan
Bryce
Caleb
Greg
John
John
Josh
Kyle
Stephen
Roberto
Christopher
Frank
Kirk
Tyler
Craig
Justin
Christopher
Kasey
Jesse
Jonathan
John
Steven
Hunter
Christopher
Craig
Tyler
Derek
Derrick
Jason
Jay
John
Kyle
Philip
Tyler
J.P.
Ben
Constantine
Ames
Lucas
Ryan
Blake
Ryan
Nick
William
Ben
West
Jeff
Chris
Bentley
Matt
Ryan
Stephen
Anthony
Chris
Frank
Jon
Michael
Rob
Tim
Jef
Arie
Sean
Chris
John
Doug
Ryan
Travis
Alejandro
Kalon
Charlie
Michael
Nathan
Stevie
Alessandro
Tony
Aaron
Kyle
Joe
Brent
David
Jackson
Jean-Paul
Lerone
Randy
Chris
Drew
Brooks
Zak
M

In [41]:
import re

last_names = []
for name in bachelorette_frame['name']:
    names = name.split(" ")
    
    
    new_names = []
    for name in names:
        cleaned_name = re.sub(r'\(\w+\)', ' ', name)
        new_names.append(cleaned_name)
    filtered = filter(lambda items: items.strip(), new_names)
    new_names = list(filtered)
        
    if len(new_names)== 1:
        last_name = "zjklafda"
    else:
        last_name = new_names[-1]
    last_names.append(last_name)
last_names

['McKee',
 'Hickl',
 'zjklafda',
 'zjklafda',
 'zjklafda',
 'M.',
 'zjklafda',
 'zjklafda',
 'R.',
 'zjklafda',
 'zjklafda',
 'zjklafda',
 'Hersh',
 'zjklafda',
 'zjklafda',
 'zjklafda',
 'Chang',
 'Clinard',
 'zjklafda',
 'zjklafda',
 'Higgins',
 "O'Quinn",
 'zjklafda',
 'zjklafda',
 'zjklafda',
 'Csincsak',
 'Mesnick',
 'Anderson',
 'Bunn',
 'Ramey',
 'Twilley',
 'Westendorf',
 'Greif',
 'Fair',
 'Brosseau',
 'Mathy',
 'Mayer',
 'Bradshaw',
 'Papachristos',
 'Hoag',
 'Winchester',
 'Fulton',
 'Capodanno',
 'Matthew',
 'Harris',
 'Konkel',
 'Hamilton',
 'Carlson',
 'Durrere',
 'Stamboulis',
 'Swiderski',
 'Locke',
 'Rosenthal',
 'Hayden',
 'Kovacs',
 'Stagliano',
 'Pavelka',
 'Pope',
 'Descant',
 'Huebner',
 'Good',
 'Barbieri',
 'Steinberg',
 'Seberhagen',
 'Fanello',
 'Petrovic',
 'Duke',
 'Hug',
 'Johnson',
 'Ambrose',
 'Duvendeck',
 'Vaderman',
 'Harlow',
 'Kuhl',
 'Bilbro',
 'Hardesty',
 'Presser',
 'Yocam',
 'Yauch',
 'Reich',
 'Martinez',
 'Lambton',
 'Neuschaefer',
 'DeWindt',

In [46]:
import re

last_names = []
for name in bachelorette_frame['name']:
    names = name.split(" ")
    
    
    new_names = []
    for name in names:
        cleaned_name = re.sub(r'\(\w+\)', ' ', name)
        new_names.append(cleaned_name)
    filtered = filter(lambda items: items.strip(), new_names)
    new_names = list(filtered)
        
    if len(new_names)== 1:
        last_name = new_names[0]
    else:
        last_name = "{} {}.".format(new_names[0], new_names[-1][0])
    last_names.append(last_name)
last_names

['Ian M.',
 'Matthew H.',
 'Chad',
 'Lanny',
 'Brad',
 'Ryan M.',
 'Sean',
 'Rick',
 'Ryan R.',
 'Todd',
 'Damon',
 'Eliot',
 'Harold H.',
 'Marcus',
 'Robert',
 'Aaron',
 'Andy C.',
 'Anselm C.',
 'Brian',
 'Chris',
 'Cory H.',
 'Jeff O.',
 'Justin',
 'Keith',
 'Trever',
 'Jesse C.',
 'Jason M.',
 'Jeremy A.',
 'Graham B.',
 'Sean R.',
 'Blaine T.',
 'Brian W.',
 'Frederick G.',
 'Robert F.',
 'Paul B.',
 'Richard M.',
 'Ronald M.',
 'Christopher B.',
 'Eric P.',
 'Ryan H.',
 'Brian W.',
 'Chandler F.',
 'Donato C.',
 'Gregory M.',
 'Jeffrey H.',
 'Jonathan K.',
 'Luke H.',
 'Patrick C.',
 'Patrick D.',
 'Spero S.',
 'Ed S.',
 'Kiptyn L.',
 'Reid R.',
 'Wes H.',
 'Jesse K.',
 'Michael S.',
 'Jake P.',
 'Tanner P.',
 'Robby D.',
 'Mark H.',
 'David G.',
 'Juan B.',
 'Mike S.',
 'Brad S.',
 'Tanner F.',
 'Sasha P.',
 'Brian D.',
 'Julien H.',
 'Mathue J.',
 'Simon A.',
 'Adam D.',
 'Bryan V.',
 'Bryce H.',
 'Caleb K.',
 'Greg B.',
 'John H.',
 'John P.',
 'Josh Y.',
 'Kyle Y.',
 'Stephe

In [44]:
l = ['james', " "]
filtered = filter(lambda items: items.strip(), l)
k = list(filtered)
k[0][0]

'j'

In [43]:
bachelorette_frame.to_csv('data/bachelorette_contestants.csv', index = False)

In [44]:
bachelor_frame.to_csv('data/bachelor_contestants.csv', index = False)